## **Importing Libraries**

In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import os

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


## **Importing Dataset**

In [3]:
data_path='/content/drive/MyDrive/Datasets/Dataset.zip'

In [4]:
with ZipFile(data_path, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

File Name                                             Modified             Size
Dataset/                                       2022-11-09 22:44:22            0
Dataset/Brain scans/                           2022-11-09 22:39:48            0
Dataset/Brain scans/No tumor/                  2022-11-09 22:39:46            0
Dataset/Brain scans/No tumor/Test/             2022-11-09 22:39:46            0
Dataset/Brain scans/No tumor/Test/no501.jpg    2021-11-14 04:39:44        12392
Dataset/Brain scans/No tumor/Test/no502.jpg    2021-11-14 04:39:44        11695
Dataset/Brain scans/No tumor/Test/no503.jpg    2021-11-14 04:39:44        42451
Dataset/Brain scans/No tumor/Test/no504.jpg    2021-11-14 04:39:44        14360
Dataset/Brain scans/No tumor/Test/no505.jpg    2021-11-14 04:39:44       127722
Dataset/Brain scans/No tumor/Test/no506.jpg    2021-11-14 04:39:44        18154
Dataset/Brain scans/No tumor/Test/no507.jpg    2021-11-14 04:39:44        11761
Dataset/Brain scans/No tumor/Test/no508.

In [5]:
def prepare_dataframes(path):
  df_train=pd.DataFrame(columns=['x','y'])
  df_test=pd.DataFrame(columns=['x','y'])
  with ZipFile(path, 'r') as zip_file:

    # Loop through the list of file names in the archive
    for file_name in zip_file.namelist():
        f_name=file_name.lower()
        # Check if the string "hello" is present in the filename
        if "mask" not in f_name:
            if 'brain' in f_name:
              if 'train' in f_name:
                new_row = {'x': file_name, 'y': '0'}
                df_train = df_train.append(new_row, ignore_index=True)
              else:
                new_row = {'x': file_name, 'y': '0'}
                df_test = df_test.append(new_row, ignore_index=True)
            elif 'breast' in f_name:
              if 'train' in f_name:
                new_row = {'x': file_name, 'y': '1'}
                df_train = df_train.append(new_row, ignore_index=True)
              else:
                new_row = {'x': file_name, 'y': '1'}
                df_test = df_test.append(new_row, ignore_index=True)
  return df_train,df_test

In [6]:
df_train,df_test=prepare_dataframes(data_path)

<ipython-input-5-950726d31d00>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(new_row, ignore_index=True)
<ipython-input-5-950726d31d00>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(new_row, ignore_index=True)
<ipython-input-5-950726d31d00>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(new_row, ignore_index=True)
<ipython-input-5-950726d31d00>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test = df_test.append(new_row, ignore_index=True)
<ipython-input-5-950726d31d00>:17: FutureWarning: The frame.append method is deprecated and will be removed 

In [7]:
df_train.head

<bound method NDFrame.head of                                                       x  y
0                   Dataset/Brain scans/No tumor/Train/  0
1            Dataset/Brain scans/No tumor/Train/no0.jpg  0
2            Dataset/Brain scans/No tumor/Train/no1.jpg  0
3           Dataset/Brain scans/No tumor/Train/no10.jpg  0
4          Dataset/Brain scans/No tumor/Train/no100.jpg  0
...                                                 ... ..
1682  Dataset/Breast scans/normal/Train/normal (95).png  1
1683  Dataset/Breast scans/normal/Train/normal (96).png  1
1684  Dataset/Breast scans/normal/Train/normal (97).png  1
1685  Dataset/Breast scans/normal/Train/normal (98).png  1
1686  Dataset/Breast scans/normal/Train/normal (99).png  1

[1687 rows x 2 columns]>

In [8]:
df_test.head

<bound method NDFrame.head of                                                      x  y
0                                 Dataset/Brain scans/  0
1                        Dataset/Brain scans/No tumor/  0
2                   Dataset/Brain scans/No tumor/Test/  0
3          Dataset/Brain scans/No tumor/Test/no501.jpg  0
4          Dataset/Brain scans/No tumor/Test/no502.jpg  0
..                                                 ... ..
306  Dataset/Breast scans/normal/Test/normal (129).png  1
307  Dataset/Breast scans/normal/Test/normal (130).png  1
308  Dataset/Breast scans/normal/Test/normal (131).png  1
309  Dataset/Breast scans/normal/Test/normal (132).png  1
310  Dataset/Breast scans/normal/Test/normal (133).png  1

[311 rows x 2 columns]>

In [9]:
df_train = df_train.sample(frac = 1)
df_test = df_test.sample(frac = 1)

## **Data Genrator**

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

t_size = (224, 224)
b_size = 32
c_mode = 'binary'
shuf = True
sed = 42
col_mode = 'rgb'

# Set up the ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# Generate the data using flow_from_dataframe()
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='x',
    y_col='y',
    target_size=t_size,
    batch_size=b_size,
    class_mode=c_mode,
    shuffle=shuf
)

# Generate the data using flow_from_dataframe()
test_generator = train_datagen.flow_from_dataframe(
    df_test,
    x_col='x',
    y_col='y',
    target_size=t_size,
    batch_size=b_size,
    class_mode=c_mode,
    shuffle=shuf
)

Found 1682 validated image filenames belonging to 2 classes.
Found 299 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 5 invalid image filename(s) in x_col="x". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 12 invalid image filename(s) in x_col="x". These filename(s) will be ignored.
  warnings.warn(


# Models

VGG16:

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import RMSprop, Adam

# Load the VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg.layers:
  layer.trainable=True

# Create a Sequential model
vgg16 = Sequential()

# Add the VGG16 model as the first layer in the Sequential model
vgg16.add(vgg)

# Add additional layers to the Sequential model
vgg16.add(Flatten())
vgg16.add(Dense(64, activation='relu'))
vgg16.add(Dense(1, activation='sigmoid'))

# Compile the model
vgg16.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

# Print the model summary
vgg16.summary()

# Train the model
vgg16.fit(train_generator,
          steps_per_epoch=train_generator.samples // b_size,
          epochs=5,
          validation_split=0.1,
          validation_data=test_generator,
          validation_steps=test_generator.samples // b_size)

# Evaluate the model on the test data
test_loss, test_accuracy = vgg16.evaluate(test_generator,
                                          steps=test_generator.samples // b_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 64)                1605696   
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 16,320,449
Trainable params: 16,320,449
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
 3/52 [>.............................] - ETA: 52:49 - loss: 0.7643 - accuracy: 0.5417

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential()

# Add convolutional layers with ReLU activation
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Add dropout to prevent overfitting
model.add(Dropout(0.25))

# Flatten the output from convolutional layers
model.add(Flatten())

# Add fully connected layers with ReLU activation
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // b_size,
                    epochs=5,
                    validation_data=test_generator,
                    validation_steps=test_generator.samples // b_size)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator,
                                          steps=test_generator.samples // b_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

# Add a convolutional layer with 32 filters, 3x3 kernel, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))

# Add a max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer with 64 filters, 3x3 kernel, and ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output to a 1D array
model.add(Flatten())

# Add a dense layer with 128 units and ReLU activation
model.add(Dense(128, activation='relu'))

# Add a dropout layer to prevent overfitting
model.add(Dropout(0.5))

# Add a final dense layer with 1 unit and sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // b_size,
                    epochs=5,
                    validation_data=test_generator,
                    validation_steps=test_generator.samples // b_size)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator,
                                          steps=test_generator.samples // b_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

In [ ]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))